In [1]:
#installing dependencies
!pip install --upgrade transformers datasets accelerate sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 86.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import pandas as pd

# Load manually labeled CSV
df = pd.read_csv("https://raw.githubusercontent.com/psabhay2003/NLP-driven-Invoice-Digitalization/refs/heads/main/labeled%20data%20sample.csv")

# Create prompt-response style training format
def make_prompt(row):
    return f"Extract invoice fields: {row['extracted_text']}"

def make_output(row):
    return f"Invoice No: {row['Invoice number']}, Date: {row['Date']}, Total Amount: {row['Total Amount']}, Vendor: {row['Vendor']}"

df['input_text'] = df.apply(make_prompt, axis=1)
df['target_text'] = df.apply(make_output, axis=1)


In [3]:
from datasets import Dataset
from transformers import T5Tokenizer

#tokenization
tokenizer = T5Tokenizer.from_pretrained("t5-base")

train_dataset = Dataset.from_pandas(df[['input_text', 'target_text']])

def preprocess(example):
    inputs = tokenizer(example['input_text'], max_length=512, truncation=True, padding="max_length")
    targets = tokenizer(example['target_text'], max_length=128, truncation=True, padding="max_length")
    inputs['labels'] = targets['input_ids']
    return inputs

tokenized_dataset = train_dataset.map(preprocess, batched=False)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [4]:
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq

#fine-tuning the T5 model
model = T5ForConditionalGeneration.from_pretrained("t5-base")

training_args = Seq2SeqTrainingArguments(
    output_dir="./t5_invoice_model",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    learning_rate=1e-4,
    num_train_epochs=8,
    eval_strategy="steps",
    eval_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    save_total_limit=2,
    fp16=True,
    logging_steps=10,
    report_to="none"
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, label_pad_token_id=-100)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset.train_test_split(test_size=0.1)['test'],
    data_collator=data_collator
)

trainer.train()

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss
50,0.036100,0.010271
100,0.020100,0.006810
150,0.018200,0.004839
200,0.015100,0.004215


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=200, training_loss=0.4107181832194328, metrics={'train_runtime': 156.6856, 'train_samples_per_second': 10.212, 'train_steps_per_second': 1.276, 'total_flos': 974332624896000.0, 'train_loss': 0.4107181832194328, 'epoch': 8.0})

In [5]:
model.save_pretrained("model")
tokenizer.save_pretrained("model")

('model/tokenizer_config.json',
 'model/special_tokens_map.json',
 'model/spiece.model',
 'model/added_tokens.json')

In [6]:
# Load OCR CSV
ocr_df = pd.read_csv("https://raw.githubusercontent.com/psabhay2003/NLP-driven-Invoice-Management-System/refs/heads/main/invoice_texts.csv")

# Prepare input prompts
ocr_df['prompt'] = ocr_df['extracted_text'].apply(lambda x: f"Extract invoice fields: {x}")

# Generate predictions for entire range of data
def generate_prediction(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(model.device)
    output = model.generate(**inputs, max_length=128)
    return tokenizer.decode(output[0], skip_special_tokens=True)

ocr_df['extracted_fields'] = ocr_df['prompt'].apply(generate_prediction)

In [7]:
ocr_df[['filename', 'extracted_fields']].to_csv("T5_output.csv", index=False)
from google.colab import files
files.download("T5_output.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
# Push this T5 output to github and then convert into the final csv which will be used in SQL database
t5_df = pd.read_csv("https://raw.githubusercontent.com/psabhay2003/NLP-driven-Invoice-Digitalization/refs/heads/main/T5_output.csv")
import re
def parse_fields(txt):
    # Use regex patterns to cover different label styles
    patterns = [
        # Standard “Invoice No: …, Date: …, Total Amount: …, Vendor: …”
        r'Invoice\s*No[:\-]?\s*(?P<inv>[^,;\n]+)[,;\n]\s*Date[:\-]?\s*(?P<date>[^,;\n]+)[,;\n]\s*Total\s*Amount[:\-]?\s*(?P<amt>[^,;\n]+)[,;\n]\s*Vendor[:\-]?\s*(?P<vend>.+)$',
        # Using “#” instead of “No”
        r'Invoice\s*#[:\-]?\s*(?P<inv>[^,;\n]+)[,;\n]\s*Date[:\-]?\s*(?P<date>[^,;\n]+)[,;\n]\s*Total[:\-]?\s*(?P<amt>[^,;\n]+)[,;\n]\s*Vendor[:\-]?\s*(?P<vend>.+)$',
        # All four as key:value pairs separated by semicolons
        r'Invoice\s*No[:\-]?\s*(?P<inv>[^;]+);\s*Date[:\-]?\s*(?P<date>[^;]+);\s*Total\s*Amount[:\-]?\s*(?P<amt>[^;]+);\s*Vendor[:\-]?\s*(?P<vend>.+)$',
        # CSV-style “1234,2023-01-01,1500,Acme Corp”
        r'^(?P<inv>[A-Z0-9\-]+)\s*,\s*(?P<date>\d{1,2}[\/\-\.][A-Za-z0-9\/\-\.]+)\s*,\s*(?P<amt>[₹\$]?\s*\d+[.,]?\d*)\s*,\s*(?P<vend>.+)$',
    ]
    for pat in patterns:
        m = re.search(pat, txt.strip(), flags=re.IGNORECASE)
        if m:
            return m.group('inv').strip(), m.group('date').strip(), m.group('amt').strip(), m.group('vend').strip()
    # Fallback: split on commas/newlines, then pick by prefix
    parts = re.split(r',|\n|;', txt)
    inv = date = amt = vend = None
    for p in parts:
        p = p.strip()
        low = p.lower()
        if inv is None and 'invoice' in low:
            inv = re.sub(r'[^A-Z0-9\-]', '', p)
        elif date is None and re.search(r'\d{1,2}[\/\-\.\s][A-Za-z]{3,}\s*\d{2,4}', p):
            date = p
        elif amt is None and re.search(r'\d+[.,]?\d*', p):
            amt = p
        elif vend is None and len(p) > 3:
            vend = p
    return inv, date, amt, vend

# Apply parsing
parsed = t5_df['extracted_fields'].apply(lambda x: pd.Series(parse_fields(str(x)),
                                                         index=['Invoice No','Date','Total Amount','Vendor']))
final = pd.concat([t5_df['filename'], parsed], axis=1)

# Save and download
final.to_csv("final_structured_output.csv", index=False)
from google.colab import files
files.download("final_structured_output.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>